# 新洋教育Kaggle零基础教学计划 - 数据挖掘项目
## 预测 Rossmann 未来的销售额

Rossmann是德国最大的日化用品超市，成立于1972年。在医药零售行业，目前Rossmann已经在7个欧洲国家拥有超过3000家药店。目前，Rossmann店铺经理的任务是**提前六周预测其日销量**。显然，商店销售受到诸多因素的影响，比如促销、竞争、假日、季节性和地点等等。 成千上万的个人经理根据各自店铺的情况预测销售量，结果的准确性可能会有很大的变化。

可靠的销售预测使商店经理能够创建有效的员工时间表，从而提高生产力和动力，比如更好的调整供应链和合理的促销策略与竞争策略，具有重要的实用价值与战略意义。 如果可以帮助Rossmann创建一个强大的预测模型，将帮助仓库管理人员专注于对他们最重要的内容：客户和团队。
因此，在这个项目中，Rossmann希望建立机器学习模型，通过给出的数据来预测德国各地1115家店铺的6周销量。

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

我们将这个notebook分为不同的步骤，你可以使用下面的链接来浏览此notebook。

* [Step 1](#step1): 导入数据
* [Step 2](#step2): 数据研究
* [Step 3](#step3): 缺失值处理
* [Step 4](#step4): 特征提取
* [Step 5](#step5): 基准模型与测试
* [Step 6](#step6): XGBoost

在该项目中包含了如下的问题：

* [问题 1](#question1): 回顾课上内容并查阅资料，归纳总结缺失值的处理方法。
* [问题 2](#question2): 这里评分标准为何采用`neg_rmspe`？
* [问题 3](#question3): 思考此时XGBoost在使用什么损失函数进行训练？

In [ ]:
# 载入必要的库
import pandas as pd
import numpy as np
import xgboost as xgb

import missingno as msno
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

<a id='step1'></a>
## 1. 导入数据

Rossmann给出的数据包含大量的特征数据，包括客户数量、假期等等。每个特征都会有对应的日销量作为标签，所以该问题为典型的监督学习问题。比赛举办方提供了4个csv文件，包括3个数据集与1个提交样本。3个数据集分别为：
- train.csv：2013/01/01至2015/07/31的1017209条历史数据，包含日销量；
- test.csv： 2015/08/01至2015/09/17的41088条历史数据，但不包含日销量；
- store.csv：1115家店铺的具体信息。

In [ ]:
# 载入数据
train = pd.read_csv('../input/rossmann-store-sales/train.csv')
test = pd.read_csv('../input/rossmann-store-sales/test.csv')
store = pd.read_csv('../input/rossmann-store-sales/store.csv')

通过``DataFrame.info()``指令可以查看DataFrame每列的数据类型以及缺失值情况。不难发现，``test.csv``和``store.csv``中均存在缺失值，这些缺失值在后续操作中都需要进行预处理。

In [ ]:
train.info(), test.info(), store.info()

<a id='step2'></a>
## 2. 数据研究

- 不难发现，当店铺关闭时，日销量必然为0。
- 去掉店铺关闭时的数据之后，再观察店铺开启时的日销量分布。可以发现日销量表现为明显的有偏分布，其偏度约为1.594，远大于0.75，因此在后续处理时需要对日销量进行对数转换。

In [ ]:
fig = plt.figure(figsize=(16,6))

ax1 = fig.add_subplot(121)
ax1.set_xlabel('Sales')
ax1.set_ylabel('Count')
ax1.set_title('Sales of Closed Stores')
plt.xlim(-1,1)
train.loc[train.Open==0].Sales.hist(align='left')

ax2 = fig.add_subplot(122)
ax2.set_xlabel('Sales')
ax2.set_ylabel('PDF')
ax2.set_title('Sales of Open Stores')
sns.distplot(train.loc[train.Open!=0].Sales)

print('The skewness of Sales is {}'.format(train.loc[train.Open!=0].Sales.skew()))

因此，我们只采用店铺营业(Open!=0)时的数据进行训练。另外，我们不采用营业时Sales==0的数据。

In [ ]:
train = train.loc[train.Open != 0]
train = train.loc[train.Sales > 0].reset_index(drop=True)

<a id='step3'></a>
## 3. 缺失值处理 

In [ ]:
# train的缺失信息：无缺失
train[train.isnull().values==True]

In [ ]:
# test的缺失信息
test[test.isnull().values==True]

In [ ]:
# store的缺失信息
msno.matrix(store)

可以看出，缺失信息集中出现在`test.csv`与`store.csv`中。下面我们对缺失值进行处理，并对特征进行合并：

In [ ]:
# 默认test中的店铺全部正常营业
test.fillna(1,inplace=True)

# 对CompetitionDistance中的缺失值采用中位数进行填补
store.CompetitionDistance = store.CompetitionDistance.fillna(store.CompetitionDistance.median())

# 对其它缺失值全部补0
store.fillna(0,inplace=True)

<a id='question1'></a>
### __问题 1:__

回顾课上内容并查阅资料，归纳总结缺失值的处理方法。

__回答:__ 


In [ ]:
# 特征合并
train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')

<a id='step4'></a>
## 4. 特征提取
### 4.1 定义特征提取函数

In [ ]:
def build_features(features, data):

    # 直接使用的特征
    features.extend(['Store','CompetitionDistance','CompetitionOpenSinceMonth','StateHoliday','StoreType','Assortment',
                     'SchoolHoliday','CompetitionOpenSinceYear', 'Promo', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear'])
    
    # 以下特征处理方式参考：https://blog.csdn.net/aicanghai_smile/article/details/80987666
    
    # 时间特征，使用dt进行处理
    features.extend(['Year','Month','Day','DayOfWeek','WeekOfYear'])
    data['Year'] = data.Date.dt.year
    data['Month'] = data.Date.dt.month
    data['Day'] = data.Date.dt.day
    data['DayOfWeek'] = data.Date.dt.dayofweek
    data['WeekOfYear'] = data.Date.dt.weekofyear
    
    # 'CompetitionOpen'：竞争对手的已营业时间
    # 'PromoOpen'：竞争对手的已促销时间
    # 两个特征的单位均为月
    features.extend(['CompetitionOpen','PromoOpen'])
    data['CompetitionOpen'] = 12*(data.Year-data.CompetitionOpenSinceYear) + (data.Month-data.CompetitionOpenSinceMonth)
    data['PromoOpen'] = 12*(data.Year-data.Promo2SinceYear) + (data.WeekOfYear-data.Promo2SinceWeek)/4.0
    data['CompetitionOpen'] = data.CompetitionOpen.apply(lambda x: x if x > 0 else 0)        
    data['PromoOpen'] = data.PromoOpen.apply(lambda x: x if x > 0 else 0)
    
    # 'IsPromoMonth'：该天店铺是否处于促销月，1表示是，0表示否
    features.append('IsPromoMonth')
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    data['monthStr'] = data.Month.map(month2str)
    data.loc[data.PromoInterval==0, 'PromoInterval'] = ''
    data['IsPromoMonth'] = 0
    for interval in data.PromoInterval.unique():
        if interval != '':
            for month in interval.split(','):
                data.loc[(data.monthStr == month) & (data.PromoInterval == interval), 'IsPromoMonth'] = 1
    
    # 字符特征转换为数字
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)
    data['StoreType'] = data['StoreType'].astype(int)
    data['Assortment'] = data['Assortment'].astype(int)
    data['StateHoliday'] = data['StateHoliday'].astype(int)

### 4.2 特征提取

In [ ]:
# 处理Date方便特征提取
train.Date = pd.to_datetime(train.Date, errors='coerce')
test.Date = pd.to_datetime(test.Date, errors='coerce')

# 使用features数组储存使用的特征
features = []

# 对train与test特征提取
build_features(features, train)
build_features([], test)

# 打印使用的特征
print(features)

<a id='step5'></a>
## 5. 基准模型与测试

### 5.1 定义评价函数

由于需要预测连续值，因此需要采用回归模型。由于该项目是Kaggle赛题，测试集是使用根均方百分比误差(Root Mean Square Percentage Error, RMSPE)评测的，因此这里只能使用RMSPE。RMSPE的计算公式为：
$${\rm RMSPE} = \frac{1}{n}\sqrt{\sum\limits_{i = 1}^n {{{\left( {\frac{{{y_i} - {{\hat y}_i}}}{{{y_i}}}} \right)}^2}}}$$
其中$y_i$与${\hat y}_i$分别为第$i$个样本标签的真实值与预测值。

In [ ]:
# 评价函数Rmspe
# 参考：https://www.kaggle.com/justdoit/xgboost-in-python-with-rmspe

def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y-yhat)**2))
    return rmspe

def rmspe_xg(yhat, y):
    y = y.get_label()
    y = np.expm1(y)
    yhat = np.expm1(yhat)
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y-yhat)**2))
    return "rmspe", rmspe

def neg_rmspe(yhat, y):
    y = np.expm1(y)
    yhat = np.expm1(yhat)
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y-yhat)**2))
    return -rmspe

### 5.2 基准测试

在上述特征基础与评价函数基础上，本文采用**决策回归树**模型进行基准测试。在代码中直接调用Sklearn中的`DecisionTreeRegressor`，配合K折交叉验证与网格搜索即可，主要调节的超参数为树的最大深度`max_depth`。

<a id='question2'></a>
### __问题 2:__

这里评分标准为何采用`neg_rmspe`？

__回答:__ 


In [ ]:
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.metrics import make_scorer

from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor(random_state=2)

cv_sets = ShuffleSplit(n_splits=5, test_size=0.2)    
params = {'max_depth':range(10,40,2)}
scoring_fnc = make_scorer(neg_rmspe)

grid = GridSearchCV(regressor,params,scoring_fnc,cv=cv_sets)
grid = grid.fit(train[features], np.log1p(train.Sales))

DTR = grid.best_estimator_

In [ ]:
# 显示最佳超参数
DTR.get_params()

In [ ]:
# 生成上传文件
submission = pd.DataFrame({"Id": test["Id"], "Sales": np.expm1(DTR.predict(test[features]))})
submission.to_csv("benchmark.csv", index=False)

模型在测试集上的Public Score为`0.18423`，Private Score为`0.22081`。下面使用XGBoost对基准测试结果进行提升。

<a id='step6'></a>
## 6. XGBoost
### 6.1 模型参数
主要调节的参数包括：
- `eta`：迭代步长；
- `max_depth`：单颗回归树的最大深度，较小导致欠拟合，较大导致过拟合；
- `subsample`：0-1之间，控制每棵树随机采样的比例，减小这个参数的值，算法会更加保守，避免过拟合。但如果这个值设置得过小，可能会导致欠拟合；
- `colsample_bytree`：0-1之间，用来控制每棵随机采样的特征的占比；
- `num_trees`：迭代步数。

In [ ]:
# 在此进行参数调节
params = {'objective': 'reg:linear',
          'eta': 0.01,
          'max_depth': 11,
          'subsample': 0.5,
          'colsample_bytree': 0.5,
          'silent': 1,
          'seed': 1
          }
num_trees = 10000

### 6.2 模型训练

In [ ]:
# 随机划分训练集与验证集
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(train, test_size=0.2, random_state=2)

dtrain = xgb.DMatrix(X_train[features], np.log1p(X_train.Sales))
dvalid = xgb.DMatrix(X_test[features], np.log1p(X_test.Sales))
dtest = xgb.DMatrix(test[features])

watchlist = [(dtrain, 'train'),(dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, early_stopping_rounds=50, feval=rmspe_xg, verbose_eval=False)

<a id='question3'></a>
### __问题 3:__

思考此时XGBoost在使用什么损失函数进行训练？

__回答:__ 


### 6.3 生产提交文件

In [ ]:
# 生成提交文件
test_probs = gbm.predict(xgb.DMatrix(test[features]), ntree_limit=gbm.best_ntree_limit)
indices = test_probs < 0
test_probs[indices] = 0
submission = pd.DataFrame({"Id": test["Id"], "Sales": np.expm1(test_probs)})
submission.to_csv("xgboost.csv", index=False)

预测结果的Public Score为`0.10932`，Private Score为`0.12051`，已非常接近Top 10%的标准线`0.11773`。